In [146]:
import os
os.environ["SPARK_HOME"] = "spark-3.5.1-bin-hadoop3"
os.environ["JAVA_HOME"] = "/usr"

In [147]:
!pip install findspark

In [148]:
import findspark
findspark.init()

In [149]:
!pip3 install pyspark==3.5.1

In [150]:
# Инициализация

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Lab2_6407_Fomin") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0") \
    .getOrCreate()

## Задание

Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

Для выполнения задания вы можете использовать любую комбинацию Spark API: **RDD API**, **Dataset API**, **SQL API**.

In [151]:
languages_path = '../data/programming-languages.csv'
posts_path = '../data/posts_sample.xml'

posts = spark.read.format('xml').options(rowTag='row').load(posts_path)

languages = spark.read \
      .option("header", True) \
      .option("inferSchema", True) \
      .option("DateTimeFormat", 'M/d/y H:m') \
      .csv(languages_path)

posts.printSchema()

root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [152]:
from pyspark.sql.functions import col

posts.select(col("_CreationDate"), col("_ViewCount"), col("_Tags")).show(5)

+--------------------+----------+--------------------+
|       _CreationDate|_ViewCount|               _Tags|
+--------------------+----------+--------------------+
|2008-08-01 02:42:...|     42817|<c#><floating-poi...|
|2008-08-01 03:08:...|     18214|<html><css><inter...|
|2008-08-01 03:17:...|      NULL|                NULL|
|2008-08-01 04:40:...|    555183|<c#><.net><datetime>|
|2008-08-01 04:55:...|    149445|<c#><datetime><ti...|
+--------------------+----------+--------------------+
only showing top 5 rows



In [153]:
from pyspark.sql.functions import split, udf, from_unixtime, unix_timestamp, lower

strip_udf = udf(lambda string: string[1: -1])
selected_posts = (
    posts.filter(col("_Tags").isNotNull())
        .withColumn("year", from_unixtime(unix_timestamp(col("_CreationDate")), "yyyy").cast("int"))
        .filter(col("year") >= 2010)
        .filter(col("year") <= 2020)
        .withColumn("views_count", col("_ViewCount"))
        .withColumn("tags", split(lower(strip_udf(col("_Tags"))), "><"))
        .select(col("year"), col("views_count"), col("tags"))
)

selected_posts.show(truncate=False)

+----+-----------+---------------------------------------------------------------+
|year|views_count|tags                                                           |
+----+-----------+---------------------------------------------------------------+
|2010|3650       |[c++, character-encoding]                                      |
|2010|617        |[sharepoint, infopath]                                         |
|2010|1315       |[iphone, app-store, in-app-purchase]                           |
|2010|973        |[symfony1, schema, doctrine, fixtures]                         |
|2010|132        |[java]                                                         |
|2010|419        |[visual-studio-2010, stylecop]                                 |
|2010|869        |[cakephp, file-upload, swfupload]                              |
|2010|1303       |[git, cygwin, putty]                                           |
|2010|748        |[drupal, drupal-6]                                             |
|201

Столбец languages распакуем по примеру
<br/>Из записи: <br/>
|2010| 599| [c#, c++, java]|
<br/>Получилось 3 новых записи:<br/>
|2010| 599| c#  |<br/>
|2010| 599| c++ |<br/>
|2010| 599| java|<br/>

In [154]:
from pyspark.sql.functions import explode
selected_posts_exploded = selected_posts.select(col("year"), col("views_count"), explode(col("tags"))).withColumnRenamed("col", "language")
selected_posts_exploded.show()

+----+-----------+------------------+
|year|views_count|          language|
+----+-----------+------------------+
|2010|       3650|               c++|
|2010|       3650|character-encoding|
|2010|        617|        sharepoint|
|2010|        617|          infopath|
|2010|       1315|            iphone|
|2010|       1315|         app-store|
|2010|       1315|   in-app-purchase|
|2010|        973|          symfony1|
|2010|        973|            schema|
|2010|        973|          doctrine|
|2010|        973|          fixtures|
|2010|        132|              java|
|2010|        419|visual-studio-2010|
|2010|        419|          stylecop|
|2010|        869|           cakephp|
|2010|        869|       file-upload|
|2010|        869|         swfupload|
|2010|       1303|               git|
|2010|       1303|            cygwin|
|2010|       1303|             putty|
+----+-----------+------------------+
only showing top 20 rows



In [155]:
from pyspark.sql.functions import lower

languages_names = languages.withColumn("name", lower(col("name"))).select(col("name"))
languages_names.show()

+------------+
|        name|
+------------+
|     a# .net|
|  a# (axiom)|
|  a-0 system|
|          a+|
|         a++|
|        abap|
|         abc|
|   abc algol|
|       abset|
|       absys|
|         acc|
|      accent|
|    ace dasl|
|        acl2|
|     act-iii|
|     action!|
|actionscript|
|         ada|
|     adenine|
|        agda|
+------------+
only showing top 20 rows



In [156]:
selected_posts_exploded = selected_posts_exploded.crossJoin(languages_names).where(col("language") == col("name"))
language_rate = selected_posts_exploded.groupBy("year", "language").agg({"views_count": "sum"}).withColumnRenamed("sum(views_count)", "views_count")
language_rate.show()

+----+-----------+-----------+
|year|   language|views_count|
+----+-----------+-----------+
|2019| typescript|       1577|
|2017|       perl|       2366|
|2012|       bash|       9333|
|2011|objective-c|     218934|
|2013|        php|     471986|
|2018|    jscript|         38|
|2013|       chef|      62421|
|2017|       curl|        897|
|2013|     delphi|      20143|
|2016|        awk|       1808|
|2016| coldfusion|       1032|
|2019|         go|        533|
|2015|       bash|      25543|
|2019|      xpath|         27|
|2013|          c|      37918|
|2016|         f#|        364|
|2014|       glsl|       2515|
|2013|       curl|      18293|
|2013|    haskell|       1043|
|2014|         f#|        457|
+----+-----------+-----------+
only showing top 20 rows



In [161]:
from pyspark.sql import Window
from pyspark.sql.functions import dense_rank

# languages_top = language_rate.orderBy(col("sum(views_count)").desc()).withColumnRenamed("sum(views_count)", "views_count")
# languages_top.show()
result = (
    language_rate
    .withColumn("rate", dense_rank().over(
        Window.partitionBy("year")
        .orderBy(col("views_count").desc())
    ))
    .where(col("rate") <= 10)
    .select("year", "language", "views_count")
    .orderBy("year", "rate")
)
result.show(100)

+----+-----------+-----------+
|year|   language|views_count|
+----+-----------+-----------+
|2010|        php|    1189629|
|2010|       java|     563211|
|2010| javascript|     316131|
|2010|objective-c|      97009|
|2010|       ruby|      76215|
|2010|          c|      66587|
|2010|     python|      60672|
|2010|     matlab|      51865|
|2010|applescript|      32305|
|2010|     delphi|      13065|
|2011| javascript|     809078|
|2011|       java|     389834|
|2011|        php|     246770|
|2011|          c|     238277|
|2011|objective-c|     218934|
|2011|     python|     203180|
|2011|       bash|      60805|
|2011|       ruby|      39223|
|2011|       perl|      28502|
|2011|     matlab|      18816|
|2012|       java|     661770|
|2012| javascript|     572750|
|2012|        php|     459506|
|2012|     python|     275337|
|2012|       ruby|     104610|
|2012|objective-c|      94438|
|2012|          c|      69276|
|2012|          r|      43917|
|2012|      xpath|      32981|
|2012|  

In [164]:
result.write.parquet("result_report.parquet")